In [1]:
import openmc
import math
from copy import deepcopy
import openmc.deplete as od

## Depletion module exploration

I will use this notebook to understand the functionality of OpenMC's ``deplete`` module towards the goal of coming up with a strategy to decouple the depletion functionality from OpenMCs transport solver.

### 1. ``Integrator``
The ``Integrator`` class is where the `integrate()` function that a user would call to run a depletion simulation lives. The `integrate()` function does several things:

1. Gets the initial nuclide concentration (``with self.operator as conc:``)
2. Gets the source rate (power or neutron flux), time step size, and step index the depletion calculation (``for i, (dt, source_rate) in enumerate(self):``)
3. Solves the transport equation (``conc, res = self._get_bos_data_from_operator(i, source_rate, conc)``)
4. Solves the bateman equations (``proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)``)
5. Saves the results in an HDF5 file.

In the following cells, I will describe how each of these steps are done in a bit more detail.

#### 1a. Obtaining initial nuclide concentration

The initial nuclide concentration is simply a pointer to the `operator` attribute of the `Integrator` object. The `operator` attribute is an instance of the `TransportOperator` class. How does this work? Let's investigate.

First, we need to create an `openmc.model` object

In [2]:
model = openmc.Model()

fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
materials = openmc.Materials([fuel, clad, water])

radii = [0.42, 0.45]
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
pin_univ = openmc.model.pin(pin_surfaces, materials)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=pin_univ, region=bound_box)
geometry = openmc.Geometry([root_cell])

settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

fuel.volume = math.pi * radii[0] ** 2


model.settings = settings
model.materials = materials
model.geometry = geometry

Next, we create an `Operator` using this model as well as a test chain file (more on this later)

In [3]:
operator = openmc.deplete.Operator(model, "../openmc/tests/chain_simple.xml")

We also will need to specify a source rate, as well as a number of depletion time steps

In [4]:
power = 174
time_steps = [30] * 6

Now we form the `Integrator` object:

In [5]:
integrator = od.PredictorIntegrator(operator, time_steps, power, timestep_units='d')

Now let's check out the use of `self.operator`. The `conc` pointer gets passed to the `_get_bos_data_from_operator` function in `abc.py` in the `bos_conc` parameter:
```python
    def _get_bos_data_from_operator(self, step_index, source_rate, bos_conc):
        """Get beginning of step concentrations, reaction rates from Operator
        """
        x = deepcopy(bos_conc)
        res = self.operator(x, source_rate)
        self.operator.write_bos_data(step_index + self._i_res)
        return x, res
```

In [6]:
x = deepcopy(integrator.operator)
integrator.operator(x, power)

TypeError: 'Operator' object is not iterable

Hmm, looks like we are getting an error. Looking at the code of `TransportOperator.__call__`, this is the correct behavior as `conc` is an `Operator` object, and not an array of floats. Let's try doing some debugging to figure out how we are getting an array of floats.

In [7]:
import pdb
pdb.set_trace()
integrator.integrate()
## b /home/ooblack/projects/openmc/openmc/deplete/abc.py:870
## p conc
## p self.operator.initial_condition()
## q

--Return--
None
> /tmp/ipykernel_7985/2721897642.py(2)<module>()
      1 import pdb
----> 2 pdb.set_trace()
      3 integrator.integrate()
      4 ## b /home/ooblack/projects/openmc/openmc/deplete/abc.py:870
      5 ## c

ipdb> b /home/ooblack/projects/openmc/openmc/deplete/abc.py:870
Breakpoint 1 at /home/ooblack/projects/openmc/openmc/deplete/abc.py:870
ipdb> c
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%

BdbQuit: 

Inspecting line 143 in `abc.py`, it looks like that the line `with self.operator as conc` executes the `__enter__` special function, and returns the inital nuclide concentration instead of an `Operator` object as one would assume by reading the code of of the `integrate()` function.

#### 1b. Source rate, timestep size, and step index
Similar to 1a, the `Integrator` class has an `__iter__` special function that returns the timesteps and source rates when passed as an argument to `enumerate()`.

#### 1c. Solving the transport equation.
The second code line of the `_get_bos_data_from_operator` function is
```python
res = self.operator(x, source_rate)
```

This instruction executes the `TransportOperator` class's `__call__` special function, which runs a transport simulation. See `Operator.__call__()` in `operator.py` for the specific implementation using the OpenMC transport solver.

The `_get_bos_data_from...` functions return the inital nuclide concentration for that depletion step (`conc`) as well as the results of the transport simulation (`res`). `res` is an instance of the `OperatorResult` named tuple.

__This means that in theory, we could build up another `TransportOperator` subclass that has an appropriate method for running another transport solver, obtaining results therefrom, and storing them in an `OperatorResult` tuple, and we would not need to change any other component.__

#### 1d. Solving the Bateman equations

The step to solve the bateman equations is a single line:

```python
                proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)
```
This instruction executes `Integrator.__call__()` special function, which is implemented within specific `Integrator` subclasses.

__My preliminary conclusion is that all of the necessary class stucture to separate out the depletion capabilties of OpenMC from its transport solver are in `abc.py`, as long as we can also find a way detangle the import statements (`openmc.lib` specifically).__

### 2. OpenMC API import statements
_Quick note: moving `TalliedFissionYieldHelper` to `helpers.py` would allow us to remove the `from openmc.lib import` statement in `abc.py`_

```Summary:
    Much of the openmc API used in the deplete module comes in the form of check_value functions.
    Critical imports
        openmc.mpi
    Code specific imports
        openmc.checkvalue
        

abc.py
    from openmc.lib import MaterialFilter, Tally
        MaterialFilter used 3 times
            line 438 (docstring), line 521 (docstring), line 535 (object)
        Tally used once
            line 532 (object)
    from openmc.checkvalue import check_type, check_greater_than
        check_type used 8 times
        check_greater_than used 4 times
        used in @setter decorators and __init__ functions
    from openmc.mpi import comm
        not explicitly used in code

atom_number.py
    from openmc import Material
        used 6 times
            line 65 (isinstance argument), the rest are in docstrings

chain.py
    from openmc.checkvalue import check_type, check_greater_than
        check_type used 5 times
            once in Chain.from_xml
            once in Chain.fission_yields
            once in Chain.validate
            twice in Chain.reduce
        check_greater_than used twice
            once in Chain.validate
            once in Chain.reduce
    from openmc.data import gnd_name, zam, DataLibrary
        gnd_name used 4 times
            3 times in replace_missing_fpy
            once in Chain.set_branch_ratios
        zam used 5 times
            once in replace_missing_fpy
            once in Chain.set_branching_ratios
            called explicitly (i.e. openmc.data.zam) in replace_missing, Chain.from_endf, and Chain.reduce
        DataLibrary used once
            line 251
    from openmc.exceptions import DataError
        DataError used once
            line 256

cram.py
    from openmc.checkvalue import check_type, check_length
        check_type used 3 times
        check_length used once
        all in IPFCramSolver.__init__

helpers.py
    from openmc.mpi import comm
        comm used 5 times
            4 in EnerergyNormalizationHelper.factor
            once in EnergyScoreHelper.reset
    from openmc.checkvalue import check_type, check_greater_than
        check_type used 4 times, all in __init__ functions
        ditto for check_greater_than
    from openmc.data import JOULE_PER_EV, REACTION_MT
        JOULE_PER_EV used once in EnergyNormalizationHelper.factor
        REACTION_MT used once in FluxCollapseHelper.generate_tallies
    from openmc.lib import (
        Tally, MaterialFilter, EnergyFilter, EnergyFunctionFilter)
        Tally used 4 times
        MaterialFilter used 3 times
        EnergyFilter used 3 times
        EnergyFunctionFilter used once
    import openmc.lib
        openmc.lib functions used:
            settings
            nuclides

operator.py
    import openmc
        openmc.Geometry used line 215
        openmc.Materials used lines 244, 264
        openmc.reset_auto_ids() used line 269, 380
    from openmc.checkvalue import check_value
        check_value used twice in Operator.__init__
    from openmc.data import DataLibrary
        DataLibrary used once in _get_nuclides_with_data
    from openmc.exceptions import DataError
        DataError used once in _find_cross_sections
    import openmc.lib
        fucntions:
            reset()
            run()
            reset_timers()
            statepoint_write()
            is_initialized
            init()
            materials on lines 591, 642
            finalize()
            keff()

    from openmc.mpi import comm
        used 17 times

nuclide.py
    from openmc.checkvalue import check_type
        used once in line 149

results.py
    import openmc
        openmc.Material on line 89
        openmc.Materials on line 513
    from openmc.mpi import comm, MPI
        comm used 10 times
        MPI used once on line 498 in call to MPI.SUM


results_list.py
    import openmc.checkvalue as cv
        used 9 times
            check_value 4
            check_type 3
            check_iterable_type 1
    from openmc.data.library import DataLibrary
        used twice in ResultsList.export_to_materials
    from openmc.material import Material, Materials
        Material used as argument to isinstance()
        Materials used twice
            both ResultsList.export_to_materials
                once is a syntax i haven't seen before in python:  def export_to_materials(self, burnup_index, nuc_with_data=None) -> Materials:
    from openmc.exceptions import DataError, InvalidArgumentError
        DataError used once in ResultsList.export_to_materials
        InvalidArgumentError not explicitly used
```

### 3. Solving the Bateman equation

Recall that the step to solve the bateman equations from the `Integrator` class is a single line:

```python
                proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)
```

However, there is much that goes on under the hood to get the output values. Let's spend some time digging into details relevant to decoupling the 

The `__call__` method of the `Integrator` class is an abstract method. It is explcitly implemented in `Integrator` subclasses located in the `integrators.py` file.

Every instance of the `__call__` function calls the `_timed_deplete` function in `Integrator`, which in turn, calls the `deplete` function from the `pool.py` file.

The different integrator classes have different approaches to doing the time integration.

1. `PredictorIntegrator` is a simple first order predictor algorithm.

2. `CECMIntegrator` is a 2-step algoorithm with a correction to the predictor step.

3. ...

The multi-step algorithms typcially make additonal `TransportOperator.__call__` calls. __This means that if we want all of the depletion algorithms to be available, we must make use of the `TransportOperator` class somehow in our transport solver independent version of the `deplete` module.__



If we are to enable a simple user-defined neutron flux so we can solve the Bateman Equation without doing transport solves, we must create machinery that do the following:

__1. An instance of the `TransportOperator` class that does not do transport solves, and instead uses a user-defined neutron flux/power array to get reaction rates and `k`. We may also want to look into non-geometric version that also accepts an array of materials.__

This may be the simplest way to get all the rest of the machinery to work together. This also has the added benefit of not needing to rely on the `openmc` C executable. All one would need to do would be to install the Python API.

__If we want to enable use of other transport solvers with the `OpenMC` deplete operators, we should write a guide on how one would go about writing a `TransportOperator` subclass for an arbitrary transport code. Special attention should be given to helping users write code for in-memory coupling (assuming their transport solver supports it). I do not think it makes sense for us to try to write a `TransportOperator` subclass for each potential transport code out there.__

### 4. List of additional considerations and features

The following is a list of features and details to consider than may be related to and build on top of this design exercise:

1. Machinery to store depletion matrices in a library; the idea is, for a specific reactor design, run a depletion simulation and then store these matrices in the library. Then, if we every want to do a depletion simulation on that reactor again, we could load the matrices from the library instead of recalculating them.

2. Decoupling the `openmc.Material` and `openmc.Material` classes and allowing for more code agnostic material handling (e.g. for use in things like ARMI, etc). An alternative to this is a way to convert external material formats to OpenMC material formats _in memory_ (several projects exist that do this but via the filesystem).